**Plan**

Question
1. Which model can best predict whether or not you will get a Kickstarter funding?
2. Which "live" projects will likely be funded?

Steps:
1. Acquire and Prep
2. Explore
    - Which features are the strongest predictors of funding? (Feature selection)
    - Out of the ones that received funding, which ones are over funded and underfunded?
3. Model
4. Evaluate

### Packages

In [7]:
import wrangle
import pandas as pd

## Acquire

Read csv file from local repo using `get_kickstarter` function from `wrangle.py`.

In [8]:
def get_kickstarter():
    df = pd.read_csv("/Users/cris/codeup-data-science/kickstarter_project/kickstarter.csv")
    return df

In [9]:
kick = get_kickstarter()

In [10]:
kick.head()

,project id,name,url,category,subcategory,location,status,goal,pledged,funded percentage,backers,funded date,levels,reward levels,updates,comments,duration
0,39409,WHILE THE TREES SLEEP,http://www.kickstarter.com/projects/emiliesaba...,Film & Video,Short Film,"Columbia, MO",successful,10500.0,11545.0,1.099524,66,"Fri, 19 Aug 2011 19:28:17 -0000",7,"$25,$50,$100,$250,$500,$1,000,$2,500",10,2,30.00
1,126581,Educational Online Trading Card Game,http://www.kickstarter.com/projects/972789543/...,Games,Board & Card Games,"Maplewood, NJ",failed,4000.0,20.0,0.005000,2,"Mon, 02 Aug 2010 03:59:00 -0000",5,"$1,$5,$10,$25,$50",6,0,47.18
2,138119,STRUM,http://www.kickstarter.com/projects/185476022/...,Film & Video,Animation,"Los Angeles, CA",live,20000.0,56.0,0.002800,3,"Fri, 08 Jun 2012 00:00:31 -0000",10,"$1,$10,$25,$40,$50,$100,$250,$1,000,$1,337,$9,001",1,0,28.00
3,237090,GETTING OVER - One son's search to finally kno...,http://www.kickstarter.com/projects/charnick/g...,Film & Video,Documentary,"Los Angeles, CA",successful,6000.0,6535.0,1.089167,100,"Sun, 08 Apr 2012 02:14:00 -0000",13,"$1,$10,$25,$30,$50,$75,$85,$100,$110,$250,$500...",4,0,32.22
4,246101,The Launch of FlyeGrlRoyalty &quot;The New Nam...,http://www.kickstarter.com/projects/flyegrlroy...,Fashion,Fashion,"Novi, MI",failed,3500.0,0.0,0.000000,0,"Wed, 01 Jun 2011 15:25:39 -0000",6,"$10,$25,$50,$100,$150,$250",2,0,30.00


**Data Cleaning**

- Kept these columns "name","category","subcategory","location","status","goal","funded percentage","backers","duration"
- Location NaN was changed to Unknown (about 1322 - too big of a sample to drop)
- Split city and state/country in location

In [ ]:
 def clean_kickstarter():
    #reduce data frame to the columns we need
    df = df[["name","category","subcategory","location","status","goal","funded percentage","backers","duration"]]
    
    df = df.location.fillna("Unknown, None", inplace=True)
    df = df[["city","state-country"]] = df["location"].str.split(',', n=1, expand=True)
    df = df.drop(["location"],axis=1)